In [19]:
import pandas as pd
import numpy as np
from pprint import pprint
import codecs
import json
import glob
pd.set_option('display.max_columns', None)
from pandas import json_normalize

In [39]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")


#Цикл для получения файла и его загрузки, используя json.load 
df_data=[]
for filename in file:
    name = filename.split("\\")[-1][:-5]
    
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
           
    for article in data['refs']:
        if article!=None:
            df_data.append([name, article[0], article[1]['day'], article[1]['month'], article[1]['time'], article[2]['rate']])
            
df_full=pd.DataFrame(data=df_data,columns=['Company', 'Post', 'day', 'month', 'time', 'rate'])

In [40]:
df_full.head()

,Company,Post,day,month,time,rate
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6
2,ESforce Holding,Из совместного исследования издателя компьютер...,29,июля,13:50,12
3,ESforce Holding,По статистике The Esports Observer за I полови...,9,июля,09:33,18
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11,апреля,12:43,11


In [41]:
#Путь к файлам .geojson
path = 'Data'
file = glob.glob(path + "/*.json")


df = pd.DataFrame(columns=['rate','subs','industries','about','Company']) 

#Цикл для получения файла и его загрузки, используя json.load 
for filename in file:
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
        name = filename.split("\\")[-1][:-5]   
        try:
            company_info=pd.concat([json_normalize(data['info']),pd.DataFrame([name],columns=['Company'])],axis=1)
        except:
            d={'rate':['Не указано'],'subs':['Не указано'],'industries':['Не указано'],'about':['Не указано']}
            company_info=pd.concat([pd.DataFrame(d),pd.DataFrame([name],columns=['Company'])],axis=1)
    df = pd.concat([df,company_info], axis=0, ignore_index=True)
df.head()    
   

,rate,subs,industries,about,Company
0,Не указано,Не указано,Не указано,Не указано,Cybersport.Metaratings
1,Не указано,Не указано,Не указано,Не указано,ESforce Holding
2,Не указано,Не указано,Не указано,Не указано,Gett
3,Не указано,Не указано,Не указано,Не указано,Goose Gaming
4,Не указано,Не указано,Не указано,Не указано,MMA.Metaratings


In [42]:
tk = df_full.merge(df, on='Company',how='left')

In [43]:
tk.shape

(1112, 10)

In [44]:
tk.head()

,Company,Post,day,month,time,rate_x,rate_y,subs,industries,about
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано
2,ESforce Holding,Из совместного исследования издателя компьютер...,29,июля,13:50,12,Не указано,Не указано,Не указано,Не указано
3,ESforce Holding,По статистике The Esports Observer за I полови...,9,июля,09:33,18,Не указано,Не указано,Не указано,Не указано
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11,апреля,12:43,11,Не указано,Не указано,Не указано,Не указано


## обработка текста

In [36]:
import pymorphy2
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Конан
[nltk_data]     Варвар\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
sw = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

def clear_text(text):
    text=text.lower()
    text = re.sub(r'[^а-яё ]','', str(text))
    tokens=word_tokenize(text, language="russian")
    tokens = [morph.parse(i)[0].normal_form for i in tokens]
    tokens = [ i for i in tokens if i not in sw and len(i) > 3]
    return tokens

In [46]:
tk['lemmatize_tokens'] = tk['Post'].apply(clear_text)

In [47]:
tk.head()

,Company,Post,day,month,time,rate_x,rate_y,subs,industries,about,lemmatize_tokens
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано,"[декабрь, состояться, церемония, награждение, ..."
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано,"[декабрь, состояться, церемония, награждение, ..."
2,ESforce Holding,Из совместного исследования издателя компьютер...,29,июля,13:50,12,Не указано,Не указано,Не указано,Не указано,"[совместный, исследование, издатель, компьютер..."
3,ESforce Holding,По статистике The Esports Observer за I полови...,9,июля,09:33,18,Не указано,Не указано,Не указано,Не указано,"[статистика, половина, впервые, самый, просмат..."
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11,апреля,12:43,11,Не указано,Не указано,Не указано,Не указано,"[компания, провести, совместный, исследование,..."


In [48]:
tk['clear_text'] = tk['lemmatize_tokens'].apply(lambda x: " ".join(x))

In [49]:
tk.head()

,Company,Post,day,month,time,rate_x,rate_y,subs,industries,about,lemmatize_tokens,clear_text
0,Cybersport.Metaratings,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано,"[декабрь, состояться, церемония, награждение, ...",декабрь состояться церемония награждение лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,Не указано,Не указано,Не указано,Не указано,"[декабрь, состояться, церемония, награждение, ...",декабрь состояться церемония награждение лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29,июля,13:50,12,Не указано,Не указано,Не указано,Не указано,"[совместный, исследование, издатель, компьютер...",совместный исследование издатель компьютерный ...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9,июля,09:33,18,Не указано,Не указано,Не указано,Не указано,"[статистика, половина, впервые, самый, просмат...",статистика половина впервые самый просматриват...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11,апреля,12:43,11,Не указано,Не указано,Не указано,Не указано,"[компания, провести, совместный, исследование,...",компания провести совместный исследование евро...


In [50]:
tk.to_csv('data.csv', index=False)

## типо векторизация...

In [51]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [52]:
tfidf = TfidfVectorizer(min_df=5,max_df=0.8, ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(tk['clear_text'])
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns = tfidf.get_feature_names())
df_tfidf.head()

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [53]:
X_tfidf

<1112x31220 sparse matrix of type '<class 'numpy.float64'>'
	with 714739 stored elements in Compressed Sparse Row format>

In [54]:
df_tfidf["Company"]=tk["Company"]

NameError: name 'df_tfidf' is not defined

## кластеризация

In [55]:
from sklearn.cluster import KMeans, Birch, MiniBatchKMeans
from sklearn.decomposition import PCA

In [56]:
model = KMeans(n_clusters=5)

In [57]:
reduced_data = PCA(n_components=2).fit_transform(X_tfidf.toarray())
model.fit_transform(reduced_data)
df_tfidf["cluster"] = model.predict(reduced_data)

C:\I\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\I\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


NameError: name 'df_tfidf' is not defined

## классификация

In [63]:
df=pd.read_json("Target.json")
df = df.rename(columns = {"Сompany":"Company"})
df

ValueError: No ':' found when decoding object value

## мдааааааа

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
x=df_tfidf.drop(['Nominations', "Company"], axis=1)
y=df_tfidf['Nominations']

NameError: name 'df_tfidf' is not defined

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42, stratify=y)

NameError: name 'x' is not defined

In [74]:
from sklearn.tree import DecisionTreeClassifier as Tree

In [75]:
tree = Tree(max_depth=20, min_samples_split=4, min_samples_leaf=2)

In [76]:
tree.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [77]:
predictions = tree.predict(x_test)

NameError: name 'x_test' is not defined

In [78]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

NameError: name 'y_test' is not defined